# K-means 1D - Implementação Sequencial


---

Este notebook tem a implementação da **etapa 0: versão sequencial** do K-means. 

 O que será feito: 
1. **Gerar os dados** de forma pseudoaleatória e reprodutiva em um formato com 1 coluna e sem cabeçalho. 
2. **Visualizar** os dados gerados.
3. **Implementar o código** C `kmeans_Id_naive.c` para a versão sequencial. 
4. **Executar** o código em C e coletar **SSE por iteração, tempo total, iterações** que servirão de baseline. 
5. **Analisar** graficamente a convergência do algoritmo. 

### 1. Geração dos dados pseudoaleatórios e reprodutivos

Será usado a biblioteca `NumPy` para gerar dados sintéticos 1D que serão usados posteriormente no K-means. O processo garante a reprodutibilidade. 

A geração consiste em: 
1. Fixar a semente aleatória `np.random.seed(24)`para garantir reprodutibilidade.
2. Criar pontos distribuídos em faixas distintas (ex: 0, 10, 20, 30) usando distribuição normal.
3. Gerar três conjuntos de teste com tamanhos crescentes:
	- Pequeno: N=10⁴ pontos, K=4 clusters
	- Médio: N=10⁵ pontos, K=8 clusters
	- Grande: N=10⁶ pontos, K=16 clusters

4. Selecionar centróides iniciais aleatoriamente dos dados gerados para cada conjunto.
5. Embaralhar os pontos usando `np.random.shuffle`
6. Converter os dados para DataFrame do pandas e salvar em arquivos CSV (1 coluna, sem cabeçalho, sem índice) usando `to_csv(index=False, header=False)`:
	- `dados_*.csv` contendo os N pontos
	- `centroides_*.csv` contendo os K centróides iniciais

In [1]:
# Importar as bibliotecas necessárias
import numpy as np
import pandas as pd
import os

# Fixar semente para reprodutibilidade
np.random.seed(24)

def gerar_dados_clusters(N, K, faixas, desvio=2.0, nome_base=''):
    """
    Gera dados 1D com K clusters em faixas distintas
    """
    pontos_por_cluster = N // K
    pontos = []
    
    print(f"\n{'='*60}")
    print(f"Gerando conjunto: {nome_base.upper()}")
    print(f"{'='*60}")
    print(f"N = {N:,} pontos")
    print(f"K = {K} clusters")
    print(f"Centros reais dos clusters: {faixas}")
    print(f"Desvio padrão: {desvio}")
    
    # Gerar pontos ao redor de cada centro
    for i, centro in enumerate(faixas):
        pontos_cluster = np.random.normal(centro, desvio, pontos_por_cluster)
        pontos.extend(pontos_cluster)
        print(f"  Cluster {i}: ~{pontos_por_cluster:,} pontos ao redor de {centro}")
    
    # Adicionar pontos restantes (se N não for divisível por K)
    pontos_restantes = N - len(pontos)
    if pontos_restantes > 0:
        centro_extra = np.random.choice(faixas)
        pontos_extras = np.random.normal(centro_extra, desvio, pontos_restantes)
        pontos.extend(pontos_extras)
    
    # Converter para array e embaralhar
    pontos = np.array(pontos)
    np.random.shuffle(pontos)
    
    # Gerar centróides iniciais (escolher K pontos aleatórios dos dados)
    indices = np.random.choice(len(pontos), K, replace=False)
    centroides_iniciais = pontos[indices]
    
    print(f"\nCentróides iniciais escolhidos: {centroides_iniciais}")
    
    # Converter para DataFrame do pandas
    df_dados = pd.DataFrame(pontos)
    df_centroides = pd.DataFrame(centroides_iniciais)
    
    # Salvar em CSV (sem índice, sem cabeçalho) - usando ../ para subir um nível
    arquivo_dados = f'../dados/dados_{nome_base}.csv'
    arquivo_centroides = f'../dados/centroides_{nome_base}.csv'
    
    df_dados.to_csv(arquivo_dados, index=False, header=False)
    df_centroides.to_csv(arquivo_centroides, index=False, header=False)
    
    print(f"\nArquivos salvos:")
    print(f"  - {arquivo_dados}")
    print(f"  - {arquivo_centroides}")
    
    return pontos, centroides_iniciais, faixas

# Conjunto pequeno: N=10^4, K=4
faixas_pequeno = [0, 10, 20, 30]
N_pequeno = 10**4
K_pequeno = 4

dados_pequeno, cent_pequeno, faixas_p = gerar_dados_clusters(
    N=N_pequeno,
    K=K_pequeno,
    faixas=faixas_pequeno,
    desvio=2.0,
    nome_base='pequeno'
)

# Conjunto médio: N=10^5, K=8
faixas_medio = [0, 10, 20, 30, 40, 50, 60, 70]
N_medio = 10**5
K_medio = 8

dados_medio, cent_medio, faixas_m = gerar_dados_clusters(
    N=N_medio,
    K=K_medio,
    faixas=faixas_medio,
    desvio=2.5,
    nome_base='medio'
)

# Conjunto grande: N=10^6, K=16
faixas_grande = [i*10 for i in range(16)]  
N_grande = 10**6
K_grande = 16

dados_grande, cent_grande, faixas_g = gerar_dados_clusters(
    N=N_grande,
    K=K_grande,
    faixas=faixas_grande,
    desvio=3.0,
    nome_base='grande'
)

print(f"\n{'='*60}")
print("Resumo dos dados gerados")
print(f"{'='*60}")
print(f"Pequeno: {N_pequeno:,} pontos, {K_pequeno} clusters")
print(f"Médio: {N_medio:,} pontos, {K_medio} clusters")
print(f"Grande: {N_grande:,} pontos, {K_grande} clusters")
print(f"\nTodos os arquivos foram salvos na pasta '../dados/'")



Gerando conjunto: PEQUENO
N = 10,000 pontos
K = 4 clusters
Centros reais dos clusters: [0, 10, 20, 30]
Desvio padrão: 2.0
  Cluster 0: ~2,500 pontos ao redor de 0
  Cluster 1: ~2,500 pontos ao redor de 10
  Cluster 2: ~2,500 pontos ao redor de 20
  Cluster 3: ~2,500 pontos ao redor de 30

Centróides iniciais escolhidos: [21.93701747 32.87325466  9.31181049 20.97084085]

Arquivos salvos:
  - ../dados/dados_pequeno.csv
  - ../dados/centroides_pequeno.csv

Gerando conjunto: MEDIO
N = 100,000 pontos
K = 8 clusters
Centros reais dos clusters: [0, 10, 20, 30, 40, 50, 60, 70]
Desvio padrão: 2.5
  Cluster 0: ~12,500 pontos ao redor de 0
  Cluster 1: ~12,500 pontos ao redor de 10
  Cluster 2: ~12,500 pontos ao redor de 20
  Cluster 3: ~12,500 pontos ao redor de 30
  Cluster 4: ~12,500 pontos ao redor de 40
  Cluster 5: ~12,500 pontos ao redor de 50
  Cluster 6: ~12,500 pontos ao redor de 60
  Cluster 7: ~12,500 pontos ao redor de 70

Centróides iniciais escolhidos: [39.47531539 69.5562491  28.

### 2. Visualização dos dados

Nesta etapa, é feita a visualização dos dados usando a biblioteca Altair para criar visualizações interativas dos três conjuntos de dados. 

A visualização consiste em:

1. Preparar os dados convertendo os arrays NumPy para DataFrames do pandas, separando pontos e centróides.

2. Aplicar jitter (dispersão aleatória) no eixo Y usando a transformação Box-Muller, pois os dados são 1D e ficariam sobrepostos. O jitter é apenas visual. 

3. Criar scatter plots com pontos azuis representando os dados, cruzes vermelhas marcando os centróides iniciais, e linha horizontal cinza como referência.

In [2]:
import altair as alt

# Configurando o tema do gráfico
def config():
    return {
        "config": {
            "background": "#09090b",
            "title": {
                "font": "SF Mono, Consolas, Monaco, monospace",
                "fontSize": 20,
                "anchor": "start",
                "color": "white"
            },
            "axis": {
                "domainColor": "white",
                "gridColor": "#444444",
                "labelFont": "SF Mono, Consolas, Monaco, monospace",
                "labelColor": "white",
                "titleFont": "SF Mono, Consolas, Monaco, monospace",
                "titleColor": "white"
            },
            "legend": {
                "labelFont": "SF Mono, Consolas, Monaco, monospace",
                "labelColor": "white",
                "titleFont": "SF Mono, Consolas, Monaco, monospace",
                "titleColor": "white"
            }
        }
    }

alt.themes.register("config", config)
alt.themes.enable("config")
alt.data_transformers.disable_max_rows()

# Preparar dados para visualização 
def preparar_dados_scatter(dados, centroides, nome):
    """
    Prepara os dados em formato adequado para scatter 
    """
    # DataFrame dos pontos 
    df_pontos = pd.DataFrame({
        'valor': dados,
        'tipo': 'Pontos',
        'conjunto': nome
    })
    
    # DataFrame dos centróides
    df_centroides = pd.DataFrame({
        'valor': centroides,
        'tipo': 'Centróides',
        'conjunto': nome
    })
    
    return df_pontos, df_centroides

# Criar DataFrames para cada conjunto 
df_p_pontos, df_p_cent = preparar_dados_scatter(dados_pequeno, cent_pequeno, 'Pequeno')
df_m_pontos, df_m_cent = preparar_dados_scatter(dados_medio, cent_medio, 'Médio')
df_g_pontos, df_g_cent = preparar_dados_scatter(dados_grande, cent_grande, 'Grande')

# Função para criar scatter 
def criar_scatter_altair(df_pontos, df_centroides, titulo, n, k):
    # Scatter plot dos pontos 
    pontos = alt.Chart(df_pontos).mark_circle(
        size=40,
        opacity=0.5,
        color='#3b82f6'  
    ).encode(
        x=alt.X('valor:Q', title='Valor', scale=alt.Scale(zero=False)),
        y=alt.Y('jitter:Q', 
                title='',
                axis=alt.Axis(labels=False, ticks=False, grid=False),
                scale=alt.Scale(domain=[-0.5, 0.5]))
    ).transform_calculate(
        jitter='sqrt(-2*log(random()))*cos(2*PI*random())*0.15'
    )
    
    # Scatter plot dos centróides
    centroides = alt.Chart(df_centroides).mark_point(
        size=400,
        shape='cross',
        color='#ef4444',  
        filled=True,
        strokeWidth=4
    ).encode(
        x=alt.X('valor:Q'),
        y=alt.Y(datum=0)
    )
    
    # Adicionar linha horizontal de referência
    linha_ref = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(
        color='#6b7280',
        strokeWidth=1,
        opacity=0.5
    ).encode(y='y:Q')
    
    # Combinar camadas
    grafico = (pontos + centroides + linha_ref).properties(
        title=f'{titulo} (N={n:,}, K={k})',
        width=500,
        height=400
    )
    
    return grafico

# Criar os 3 gráficos
grafico_pequeno = criar_scatter_altair(
    df_p_pontos, 
    df_p_cent,
    'Conjunto Pequeno',
    N_pequeno,
    K_pequeno
)

grafico_medio = criar_scatter_altair(
    df_m_pontos,
    df_m_cent,
    'Conjunto Médio',
    N_medio,
    K_medio
)

grafico_grande = criar_scatter_altair(
    df_g_pontos,
    df_g_cent,
    'Conjunto Grande',
    N_grande,
    K_grande
)

# Concatenar horizontalmente os 3 gráficos
grafico_final = grafico_pequeno | grafico_medio | grafico_grande

# Exibir o gráfico
grafico_final


### 3: Implementação k-means sequencial

Implementação do código `kmeans_1d_naive.c`com o uso `%%writefile%%`. 

In [3]:
%%writefile kmeans_1d_naive.c
/* kmeans_1d_naive.c
   K-means 1D (C99), implementação "naive":
   - Lê X (N linhas, 1 coluna) e C_init (K linhas, 1 coluna) de CSVs sem cabeçalho
   - Itera assignment + update até max_iter ou variação relativa do SSE < eps.
   - Salva (opcional) assign (N linhas) e centróides finais (K linhas).
   - Salva evolução do SSE em arquivo CSV para análise de convergência.
   
   Compilar: gcc -O2 -std=c99 kmeans_1d_naive.c -o kmeans_1d_naive -lm
   Uso:      ./kmeans_1d_naive dados.csv centroides_iniciais.csv [max_iter=50] [eps=1e-4] [assign.csv] [centroids.csv] [sse_evolution.csv]
*/
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

/* ---------- util CSV 1D: cada linha tem 1 número ---------- */
static int count_rows(const char *path){
    FILE *f = fopen(path, "r");
    if(!f){ fprintf(stderr,"Erro ao abrir %s\n", path); exit(1); }
    int rows=0; char line[8192];
    while(fgets(line,sizeof(line),f)){
        int only_ws=1;
        for(char *p=line; *p; p++){
            if(*p!=' ' && *p!='\t' && *p!='\n' && *p!='\r'){ only_ws=0; break; }
        }
        if(!only_ws) rows++;
    }
    fclose(f);
    return rows;
}

static double *read_csv_1col(const char *path, int *n_out){
    int R = count_rows(path);
    if(R<=0){ fprintf(stderr,"Arquivo vazio: %s\n", path); exit(1); }
    double *A = (double*)malloc((size_t)R * sizeof(double));
    if(!A){ fprintf(stderr,"Sem memoria para %d linhas\n", R); exit(1); }
    FILE *f = fopen(path, "r");
    if(!f){ fprintf(stderr,"Erro ao abrir %s\n", path); free(A); exit(1); }
    char line[8192];
    int r=0;
    while(fgets(line,sizeof(line),f)){
        int only_ws=1;
        for(char *p=line; *p; p++){
            if(*p!=' ' && *p!='\t' && *p!='\n' && *p!='\r'){ only_ws=0; break; }
        }
        if(only_ws) continue;
        /* aceita vírgula/ponto-e-vírgula/espaco/tab, pega o primeiro token numérico */
        const char *delim = ",; \t";
        char *tok = strtok(line, delim);
        if(!tok){ fprintf(stderr,"Linha %d sem valor em %s\n", r+1, path); free(A); exit(1); }
        A[r] = atof(tok);
        r++;
        if(r>R) break;
    }
    fclose(f);
    *n_out = R;
    return A;
}

static void write_assign_csv(const char *path, const int *assign, int N){
    if(!path) return;
    FILE *f = fopen(path, "w");
    if(!f){ fprintf(stderr,"Erro ao abrir %s para escrita\n", path); return; }
    for(int i=0;i<N;i++) fprintf(f, "%d\n", assign[i]);
    fclose(f);
}

static void write_centroids_csv(const char *path, const double *C, int K){
    if(!path) return;
    FILE *f = fopen(path, "w");
    if(!f){ fprintf(stderr,"Erro ao abrir %s para escrita\n", path); return; }
    for(int c=0;c<K;c++) fprintf(f, "%.6f\n", C[c]);
    fclose(f);
}

/* ---------- k-means 1D ---------- */

/* assignment: para cada X[i], encontra c com menor (X[i]-C[c])^2 */
static double assignment_step_1d(const double *X, const double *C, int *assign, int N, int K){
    double sse = 0.0;
    for(int i=0;i<N;i++){
        int best = -1;
        double bestd = 1e300;
        for(int c=0;c<K;c++){
            double diff = X[i] - C[c];
            double d = diff*diff;
            if(d < bestd){ bestd = d; best = c; }
        }
        assign[i] = best;
        sse += bestd;
    }
    return sse;
}

/* update: média dos pontos de cada cluster (1D)
   se cluster vazio, copia X[0] (estratégia naive) */
static void update_step_1d(const double *X, double *C, const int *assign, int N, int K){
    double *sum = (double*)calloc((size_t)K, sizeof(double));
    int *cnt = (int*)calloc((size_t)K, sizeof(int));
    if(!sum || !cnt){ fprintf(stderr,"Sem memoria no update\n"); exit(1); }
    for(int i=0;i<N;i++){
        int a = assign[i];
        cnt[a] += 1;
        sum[a] += X[i];
    }
    for(int c=0;c<K;c++){
        if(cnt[c] > 0) C[c] = sum[c] / (double)cnt[c];
        else           C[c] = X[0]; /* simples: cluster vazio recebe o primeiro ponto */
    }
    free(sum); free(cnt);
}

static void kmeans_1d(const double *X, double *C, int *assign,
                      int N, int K, int max_iter, double eps,
                      int *iters_out, double *sse_out, const char *sse_path)
{
    // Abrir arquivo para salvar evolução do SSE
    FILE *sse_file = NULL;
    if(sse_path){
        sse_file = fopen(sse_path, "w");
        if(sse_file) fprintf(sse_file, "iteration,sse\n");
    }
    
    double prev_sse = 1e300;
    double sse = 0.0;
    int it;
    
    for(it=0; it<max_iter; it++){
        sse = assignment_step_1d(X, C, assign, N, K);
        
        // Salvar SSE desta iteração
        if(sse_file) fprintf(sse_file, "%d,%.6f\n", it, sse);
        
        /* parada por variação relativa do SSE */
        double rel = fabs(sse - prev_sse) / (prev_sse > 0.0 ? prev_sse : 1.0);
        if(rel < eps){ it++; break; }
        
        update_step_1d(X, C, assign, N, K);
        prev_sse = sse;
    }
    
    // Fechar arquivo de evolução do SSE
    if(sse_file) fclose(sse_file);
    
    *iters_out = it;
    *sse_out = sse;
}

/* ---------- main ---------- */

int main(int argc, char **argv){
    if(argc < 3){
        printf("Uso: %s dados.csv centroides_iniciais.csv [max_iter=50] [eps=1e-4] [assign.csv] [centroids.csv] [sse_evolution.csv]\n", argv[0]);
        printf("Obs: arquivos CSV com 1 coluna (1 valor por linha), sem cabeçalho\n");
        return 1;
    }
    const char *pathX = argv[1];
    const char *pathC = argv[2];
    int max_iter = (argc>3)? atoi(argv[3]) : 50;
    double eps   = (argc>4)? atof(argv[4]) : 1e-4;
    const char *outAssign   = (argc>5)? argv[5] : NULL;
    const char *outCentroid = (argc>6)? argv[6] : NULL;
    const char *outSSE      = (argc>7)? argv[7] : NULL;
    
    if(max_iter <= 0 || eps <= 0.0){
        fprintf(stderr,"Parâmetros inválidos: max_iter>0 e eps>0\n");
        return 1;
    }
    
    int N=0, K=0;
    double *X = read_csv_1col(pathX, &N);
    double *C = read_csv_1col(pathC, &K);
    int *assign = (int*)malloc((size_t)N * sizeof(int));
    if(!assign){ fprintf(stderr,"Sem memoria para assign\n"); free(X); free(C); exit(1); }
    
    clock_t t0 = clock();
    int iters = 0; double sse = 0.0;
    kmeans_1d(X, C, assign, N, K, max_iter, eps, &iters, &sse, outSSE);
    clock_t t1 = clock();
    
    double ms = 1000.0 * (double)(t1 - t0) / (double)CLOCKS_PER_SEC;
    
    printf("K-means 1D (naive)\n");
    printf("N=%d K=%d max_iter=%d eps=%g\n", N, K, max_iter, eps);
    printf("Iterações: %d | SSE final: %.6f | Tempo: %.1f ms\n", iters, sse, ms);
    
    write_assign_csv(outAssign, assign, N);
    write_centroids_csv(outCentroid, C, K);
    
    free(assign); free(X); free(C);
    return 0;
}


Overwriting kmeans_1d_naive.c


### 4: Compilação do k-means sequencial

Agora com os dados gerados e o arquivo C gerado, é possível executar o k-means.

A **compilação** é feita usando o `GCC` com as flags `02` para uma medição justa e `-lm` para vincular a biblioteca matemática e o padrão é específicado com `-std=c99`. 

Já a `execução` é feita para três conjuntos de dados distintos, passando os seguintes parâmetros: 

- `dados_[tamanho].csv`: Arquivos com os pontos de dados (10k, 100k e 1M pontos).
- `centroides_[tamanho].csv`: Centróides iniciais (4, 8 e 16 centróides respectivamente).
- `100`: max_iter (número máximo de iterações).
- `1e-6`: eps (critério de parada por convergência, o algoritmo para quando a variação relativa do SSE é menor que este valor).
- `assign_[tamanho].csv`: Arquivo de saída com as atribuições de cluster para cada ponto.
- `centroids_[tamanho].csv`: Arquivo de saída com os centróides finais após convergência.
- `sse_evolution_[tamanho].csv`: Arquivo que registra a evolução do SSE a cada iteração.

O programa imprime o **número de iterações, SSE final e tempo de execução em milissegundos**. Estes valores servirão como baseline para calcular o speedup das versões paralelizadas (OpenMP, CUDA e MPI).


In [4]:
%%bash

# Criar pasta de resultados 
echo "Criando pasta de resultados..."
mkdir -p resultados

# Compilar o código sequencial
echo ""
echo "Compilando o código K-means sequencial..."
gcc -O2 -std=c99 kmeans_1d_naive.c -o kmeans_1d_naive -lm
echo "Compilação concluída!"

# Executar conjunto pequeno
echo ""
echo "Executando K-means: Conjunto pequeno"
./kmeans_1d_naive ../dados/dados_pequeno.csv ../dados/centroides_pequeno.csv 100 1e-6 resultados/assign_pequeno.csv resultados/centroids_pequeno.csv resultados/sse_evolution_pequeno.csv

# Executar conjunto médio
echo ""
echo "Executando K-means: Conjunto MÉDIO"
./kmeans_1d_naive ../dados/dados_medio.csv ../dados/centroides_medio.csv 100 1e-6 resultados/assign_medio.csv resultados/centroids_medio.csv resultados/sse_evolution_medio.csv

# Executar conjunto grande
echo ""
echo "Executando K-means: Conjunto GRANDE"
./kmeans_1d_naive ../dados/dados_grande.csv ../dados/centroides_grande.csv 100 1e-6 resultados/assign_grande.csv resultados/centroids_grande.csv resultados/sse_evolution_grande.csv

echo ""
echo "Execução completa. Resultados salvos em: resultados/"

Criando pasta de resultados...

Compilando o código K-means sequencial...
Compilação concluída!

Executando K-means: Conjunto pequeno
K-means 1D (naive)
N=10000 K=4 max_iter=100 eps=1e-06
Iterações: 11 | SSE final: 38138.964917 | Tempo: 1.3 ms

Executando K-means: Conjunto MÉDIO
K-means 1D (naive)
N=100000 K=8 max_iter=100 eps=1e-06
Iterações: 10 | SSE final: 552187.892633 | Tempo: 12.1 ms

Executando K-means: Conjunto GRANDE
K-means 1D (naive)
N=1000000 K=16 max_iter=100 eps=1e-06
Iterações: 100 | SSE final: 8279161.144560 | Tempo: 2100.1 ms

Execução completa. Resultados salvos em: resultados/


### 5. Analisar graficamente a convergência do algoritmo. 

Para visualizar a convergência dos diferentes conjuntos de dados, foi criado um gráfico de linha que mostra a evolução do SSE a cada iteraçao para os três conjuntos de dados (pequeno, médio e grande).

In [5]:
# Ler os arquivos de evolução do SSE
df_pequeno = pd.read_csv('resultados/sse_evolution_pequeno.csv')
df_pequeno['conjunto'] = 'Pequeno (N=10k, K=4)'

df_medio = pd.read_csv('resultados/sse_evolution_medio.csv')
df_medio['conjunto'] = 'Médio (N=100k, K=8)'

df_grande = pd.read_csv('resultados/sse_evolution_grande.csv')
df_grande['conjunto'] = 'Grande (N=1M, K=16)'

# Combinar os três conjuntos
df_all = pd.concat([df_pequeno, df_medio, df_grande], ignore_index=True)

# Criar gráfico 
convergencia = alt.Chart(df_all).mark_line(
    point=True,
    strokeWidth=3
).encode(
    x=alt.X('iteration:Q', title='Iteração'),
    y=alt.Y('sse:Q', title='SSE (Sum of Squared Errors)'),
    color=alt.Color('conjunto:N', 
                    title='Conjunto de Dados',
                    scale=alt.Scale(range=['#3b82f6', '#10b981', '#f59e0b'])),
).properties(
    title='Convergência do K-means: Evolução do SSE por Iteração',
    width=1000,
    height=600
)

# Exibir gráfico
convergencia


/home/gui/miniconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)